In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from os import system
import os
import keras
import util
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from sklearn.cluster import KMeans

In [2]:
import xml.etree.ElementTree as ET
import classification_starter as start
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

import util

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from gensim import corpora, models, similarities
import gensim

In [4]:
labels = []
with open('t_train_starter.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        labels.append(row)
        
sentences = []
sent_lengths = []
with open('sentences.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sent_lengths.append(len(row))
        sentences.append(row)

In [26]:
np.array(labels[0]).astype(int)

array([ 8,  6, 12, ...,  8,  8,  3])

In [ ]:
sentences_test = []

with open('sentences_test.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:

        sentences_test.append(row)

In [236]:
        
sentences_valid = []
sent_lengths_valid = []
with open('sentences_valid.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sent_lengths_valid.append(len(row))
        sentences_valid.append(row)

In [237]:
ls = len(sentences)
lv = len(sentences_valid)
y_train = np.array(labels[0][:ls]).astype(int)
y_valid = np.array(labels[0][ls:ls+lv]).astype(int)

In [ ]:
system("say I am finished")

In [ ]:
# Set values for various parameters
num_features = 5000    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 6       # Number of threads to run in parallel
context = 15          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

In [8]:
model.wv.syn0.shape

(28774, 700)

In [ ]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 150

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )


# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [ ]:
from os import system
system("say I am finished")

In [13]:
# For the first 3 clusters
for cluster in xrange(0,3):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
['"C:\\WINDOWS\\Microsoft.NET\\Framework\\v2.0.50727\\config\\enterprisesec.config.cch"', '"18467-41"', '"AC3D6A4&amp;id', '"HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\DirectDraw\\Compatibility\\Rogue', '"00:07.562"', '"1188"', 'Einstellungen\\Administrator\\Startmen&#252;\\Programme\\Zubeh&#246;r"', '"c:\\rich\\03\\images\\"', 'Dateien\\SpeechEngines\\Microsoft\\TTS"', 'website_So"', 'Type\\application/xml\\Bits"', '"......(...Y5J&lt;A%"', '"C:\\WINDOWS\\assembly\\NativeImages_v2.0.50727_32\\System.ServiceProce#\\*.*"', '"str133"', '"329"', '"HKEY_CURRENT_USER\\Software\\Microsoft\\CTF\\TIP"', '"$76BB4D48"', '426FFAA34"', 'Manager\\iefdm2.dll"', 'DB\\msdasql.dll"', '&quot;6410965"', '"HKEY_LOCAL_MACHINE\\SOFTWARE\\Classes\\PROTOCOLS\\Handler\\about"', '"url11"', '"{FAEDCF69-31FE-11D1-AAD2-00805FC1270E}"', '"Global\\{4A0A574F-EC90-4A3F-8C4E-BA240F5991E9}"', '1997"', 'Media\\WMSDK\\General"', '"HOMSDLVT"', '"ShowEffects"', '"$441E19F8"', '"..Accept-Ranges:"', 'nature"', '4', '"

In [14]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    num_centroids = max( word_centroid_map.values() ) + 1

    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )

    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1

    return bag_of_centroids

In [92]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (len(sentences), num_clusters), \
    dtype="float32" )

# Transform the training set into bags of centroids
counter = 0
for sentence in sentences:
    train_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1

In [93]:
test_centroids = np.zeros(( 3724, num_clusters), \
    dtype="float32" )

counter = 0
with open('sentences_test.csv') as csvfile:
    reader = csv.reader(csvfile)
    for sentence in reader:
        test_centroids[counter] = create_bag_of_centroids( sentence, \
            word_centroid_map )
        counter += 1
    
train_centroids=train_centroids.astype(int)
test_centroids=test_centroids.astype(int)

In [244]:
valid_centroids = np.zeros(( len(sentences_valid), num_clusters), \
    dtype="float32" )

counter = 0
for sentence in sentences_valid:
    valid_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1
    
train_centroids=train_centroids.astype(int)
valid_centroids=valid_centroids.astype(int)

In [ ]:
sentences_test = []

with open('sentences_test.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:

        sentences_test.append(row)

In [18]:
print test_centroids.shape
print train_centroids.shape

(3724, 57)
(3086, 57)


In [194]:
valid_centroids

array([[ 6935, 11299,   115, ...,   133,   740,    12],
       [   75,     0,    21, ...,    91,    20,     7],
       [  112,     0,    19, ...,    91,    20,     7],
       ..., 
       [   27,     0,     4, ...,     0,     7,     2],
       [  114,     0,    20, ...,    91,    20,     7],
       [   21,     0,     4, ...,     0,     7,     2]])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train,15)
y_valid = to_categorical(y_valid,15)

In [233]:
net = Sequential()
net.add(Dense(57, input_dim=200))

net.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [234]:
net.fit(train_centroids, y_train, nb_epoch=5)

ValueError: Error when checking model input: expected dense_input_11 to have shape (None, 200) but got array with shape (200, 57)

In [235]:
print train_centroids.shape
print y_train.shape

(200, 15)


In [56]:
ids = pd.read_csv("test_ids.csv")
ids

0
0     0015c8c9ff02fea9d0f45692b9eebfb4abff4e42f
1     001f298a534ae4b0db7f2707169250aa215c3b5f2
2     001f5fdaaa8bbe20303527198d09a30bb7ca3eb50
3     002ca2c41b649f85c05ae30013436781a932fecc6
4     003e109543b4ea22d2bcc1ec309bf2fd34e9a1a1d
5     004070b468d6bb29848c76cfCd5887849c7bb648d
6     00461dd05c981edde167a5947c365472141e04bb1
7     005b95d2520C8621171566f5803437b0c443778e1
8     0071a3b818ed06d3865a24fdb31d4147c67fabfc5
9     007436715ec13cedd38344772a2144a3d79f3ea68
10    00810ef95f86daee349107255601afc51e9a7eba2
11    00888535488746a0c494c41fbe93940f72aad0f30
12    008cc72e6d82f36c8d4a95ab35e241b5e58411845
13    00a2b881ddba2fbece1d2853bc8484072c0ad795b
14    00ab51519cdf57cf9a53aa4e0caeada0753a36bd4
15    00c0646c7dfd525f704f0f81dfa9f900e3357abf6
16    00e379c843a3f67fb8d4A7832c0701d797f11f9f3
17    00e586a3c25d97bc1b112cbad672a91ba238f4296
18    00fb3b50ccfadd5b427aba20792bb95a4260db685
19    010d1e20d98e8b87b4208a6b5c80f0C1c088037d8
20    0113a8aead089b16a65f837b3524f58e84c34e303
21    01178cc12a1c4340c520ce669728Ad3538ab2f0f1
22    013652ef9219818cbfb498aaddb30c9774cdce199
23    01392e526a68f01285549de36e16333e95e51ce60
24    01457a2418244506c274051df725c475d1f7d92d7
25    0173a0d4005e410835b0fD49e3f02268265c3f8bb
26    0175d1a73db8e4c2fd406628978721b2664d2deb8
27    01d9d46e124041ac3e98E4fd9fea0ee0e838e3c15
28    01ea0141110652ac0ecee45b753877bCe66a887cc
29    02049c8003b0c4ef4f7524c0cb8add4a2fe78bc8f
...                                         ...
3694  fd76e9af95e779a5f23e2791474f3edBf04e87a01
3695  fd875a4fb7aaba3e643a2db3692d018871e04fd76
3696  fd9a2393f610e53a43043bd019cc696f5c07e4944
3697  fdbbeffe82dbef183af9ffb404e89500d6bc0b6c3
3698  fdbfd653c850dfd6c4aea152f6500c52d5950e43d
3699  fdd14bdbd3f1aac3116c2c1169a86cbc61b1Ae9c1
3700  fdd658beec58b195a2c2edc5d92ddfca77285b559
3701  fde0cfe1e6656cd418c3f8a4e0ad47c4c17e25f34
3702  fdf35b62e7bd49fdf50bb2143bbaaf7121520ed65
3703  fe09e746b753fcee22e2c7d2be2883685761d4145
3704  fe255d4daf56461e5D7a5749a054eedc50b087509
3705  fe4e28d7b30d26a2fde0fc4c815e7eb0a52367fdd
3706  fe53725f47540d4f31d61bdd0a5aa5bb5cca28a50
3707  fe7887a3ff58c609fe633b6118e0c033a92ffd555
3708  feae6E291a49da14d05a6ffb9fbee2815d7c8153d
3709  fec6d2Eb64b083c0b8f4b4f0bd04671e78607be09
3710  fec8e57b083bd354dd6b02d129baae82f7591eae6
3711  fed5e724a012c38ec624fba4fd3650da59a5595c5
3712  feea3c1b728377170a6ad93689f0d9b2c07b0fe12
3713  ff15577c4d58442b758e85670263ada014fc91dd7
3714  ff272446010174efce45afa0e3ec77acba122b6df
3715  ff31496ea9aa36804397160dcc442890ec3a7df95
3716  ff393ca012e3bb6b89846114181ece70fde991c58
3717  ff3f10990b37a97c4990cfa36c14A6feb79c5ed32
3718  ff42c4ecb06a8838228a03cb930e87e3bdf5c08e2
3719  ff5abc2369a9Bea144b697fa2c147f37f8bbcc410
3720  ff7cc6e8d270894363730c008d579197bebcbd07d
3721  ff85866b215233b2fecdca2c2b8fda54ad24c86fd
3722  ff884224571e0476990574df5da76e0991db583af
3723  ffc47163a530c51ef2e6572d786aefbaed99890f2

[3724 rows x 1 columns]

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 200)

y_train = np.array(labels[0]).astype(int)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,y_train)
result = forest.predict(test_centroids)

In [103]:
df_res = pd.DataFrame(ids)
df_res["Prediction"] = result
df_res.columns = ['Id','Prediction']
df_res

Id  Prediction
0     0015c8c9ff02fea9d0f45692b9eebfb4abff4e42f           8
1     001f298a534ae4b0db7f2707169250aa215c3b5f2          10
2     001f5fdaaa8bbe20303527198d09a30bb7ca3eb50          13
3     002ca2c41b649f85c05ae30013436781a932fecc6           8
4     003e109543b4ea22d2bcc1ec309bf2fd34e9a1a1d           8
5     004070b468d6bb29848c76cfCd5887849c7bb648d          10
6     00461dd05c981edde167a5947c365472141e04bb1           5
7     005b95d2520C8621171566f5803437b0c443778e1           8
8     0071a3b818ed06d3865a24fdb31d4147c67fabfc5          12
9     007436715ec13cedd38344772a2144a3d79f3ea68           8
10    00810ef95f86daee349107255601afc51e9a7eba2           8
11    00888535488746a0c494c41fbe93940f72aad0f30           8
12    008cc72e6d82f36c8d4a95ab35e241b5e58411845           9
13    00a2b881ddba2fbece1d2853bc8484072c0ad795b          10
14    00ab51519cdf57cf9a53aa4e0caeada0753a36bd4          10
15    00c0646c7dfd525f704f0f81dfa9f900e3357abf6           8
16    00e379c843a3f67fb8d4A7832c0701d797f11f9f3          12
17    00e586a3c25d97bc1b112cbad672a91ba238f4296          10
18    00fb3b50ccfadd5b427aba20792bb95a4260db685          11
19    010d1e20d98e8b87b4208a6b5c80f0C1c088037d8           8
20    0113a8aead089b16a65f837b3524f58e84c34e303           8
21    01178cc12a1c4340c520ce669728Ad3538ab2f0f1          12
22    013652ef9219818cbfb498aaddb30c9774cdce199           8
23    01392e526a68f01285549de36e16333e95e51ce60           2
24    01457a2418244506c274051df725c475d1f7d92d7           8
25    0173a0d4005e410835b0fD49e3f02268265c3f8bb          12
26    0175d1a73db8e4c2fd406628978721b2664d2deb8           8
27    01d9d46e124041ac3e98E4fd9fea0ee0e838e3c15           8
28    01ea0141110652ac0ecee45b753877bCe66a887cc           8
29    02049c8003b0c4ef4f7524c0cb8add4a2fe78bc8f           3
...                                         ...         ...
3694  fd76e9af95e779a5f23e2791474f3edBf04e87a01           8
3695  fd875a4fb7aaba3e643a2db3692d018871e04fd76          10
3696  fd9a2393f610e53a43043bd019cc696f5c07e4944           7
3697  fdbbeffe82dbef183af9ffb404e89500d6bc0b6c3           8
3698  fdbfd653c850dfd6c4aea152f6500c52d5950e43d           9
3699  fdd14bdbd3f1aac3116c2c1169a86cbc61b1Ae9c1           0
3700  fdd658beec58b195a2c2edc5d92ddfca77285b559           9
3701  fde0cfe1e6656cd418c3f8a4e0ad47c4c17e25f34           1
3702  fdf35b62e7bd49fdf50bb2143bbaaf7121520ed65           8
3703  fe09e746b753fcee22e2c7d2be2883685761d4145          10
3704  fe255d4daf56461e5D7a5749a054eedc50b087509          10
3705  fe4e28d7b30d26a2fde0fc4c815e7eb0a52367fdd          10
3706  fe53725f47540d4f31d61bdd0a5aa5bb5cca28a50           8
3707  fe7887a3ff58c609fe633b6118e0c033a92ffd555           8
3708  feae6E291a49da14d05a6ffb9fbee2815d7c8153d           8
3709  fec6d2Eb64b083c0b8f4b4f0bd04671e78607be09           8
3710  fec8e57b083bd354dd6b02d129baae82f7591eae6           8
3711  fed5e724a012c38ec624fba4fd3650da59a5595c5           8
3712  feea3c1b728377170a6ad93689f0d9b2c07b0fe12          12
3713  ff15577c4d58442b758e85670263ada014fc91dd7           8
3714  ff272446010174efce45afa0e3ec77acba122b6df          10
3715  ff31496ea9aa36804397160dcc442890ec3a7df95          10
3716  ff393ca012e3bb6b89846114181ece70fde991c58           8
3717  ff3f10990b37a97c4990cfa36c14A6feb79c5ed32           8
3718  ff42c4ecb06a8838228a03cb930e87e3bdf5c08e2           8
3719  ff5abc2369a9Bea144b697fa2c147f37f8bbcc410          14
3720  ff7cc6e8d270894363730c008d579197bebcbd07d           8
3721  ff85866b215233b2fecdca2c2b8fda54ad24c86fd          12
3722  ff884224571e0476990574df5da76e0991db583af           8
3723  ffc47163a530c51ef2e6572d786aefbaed99890f2           8

[3724 rows x 2 columns]

In [48]:
ids = pd.read_csv("train_ids_starter.csv",header=None)

In [ ]:
ids = np.array(ids)

In [104]:
df_res.to_csv("rand_forest_preds2.csv",index=None)

In [88]:
p = pd.read_csv("rand_forest_preds1.csv")
p

Id  Prediction
0     0015c8c9ff02fea9d0f45692b9eebfb4abff4e42f           8
1     001f298a534ae4b0db7f2707169250aa215c3b5f2          10
2     001f5fdaaa8bbe20303527198d09a30bb7ca3eb50           8
3     002ca2c41b649f85c05ae30013436781a932fecc6           8
4     003e109543b4ea22d2bcc1ec309bf2fd34e9a1a1d           8
5     004070b468d6bb29848c76cfCd5887849c7bb648d          10
6     00461dd05c981edde167a5947c365472141e04bb1           5
7     005b95d2520C8621171566f5803437b0c443778e1           8
8     0071a3b818ed06d3865a24fdb31d4147c67fabfc5          12
9     007436715ec13cedd38344772a2144a3d79f3ea68           8
10    00810ef95f86daee349107255601afc51e9a7eba2           8
11    00888535488746a0c494c41fbe93940f72aad0f30           8
12    008cc72e6d82f36c8d4a95ab35e241b5e58411845           9
13    00a2b881ddba2fbece1d2853bc8484072c0ad795b          10
14    00ab51519cdf57cf9a53aa4e0caeada0753a36bd4          10
15    00c0646c7dfd525f704f0f81dfa9f900e3357abf6           8
16    00e379c843a3f67fb8d4A7832c0701d797f11f9f3          12
17    00e586a3c25d97bc1b112cbad672a91ba238f4296          10
18    00fb3b50ccfadd5b427aba20792bb95a4260db685          11
19    010d1e20d98e8b87b4208a6b5c80f0C1c088037d8           8
20    0113a8aead089b16a65f837b3524f58e84c34e303           8
21    01178cc12a1c4340c520ce669728Ad3538ab2f0f1          12
22    013652ef9219818cbfb498aaddb30c9774cdce199           8
23    01392e526a68f01285549de36e16333e95e51ce60           2
24    01457a2418244506c274051df725c475d1f7d92d7           8
25    0173a0d4005e410835b0fD49e3f02268265c3f8bb          12
26    0175d1a73db8e4c2fd406628978721b2664d2deb8           8
27    01d9d46e124041ac3e98E4fd9fea0ee0e838e3c15           8
28    01ea0141110652ac0ecee45b753877bCe66a887cc           8
29    02049c8003b0c4ef4f7524c0cb8add4a2fe78bc8f           3
...                                         ...         ...
3694  fd76e9af95e779a5f23e2791474f3edBf04e87a01           0
3695  fd875a4fb7aaba3e643a2db3692d018871e04fd76          10
3696  fd9a2393f610e53a43043bd019cc696f5c07e4944           7
3697  fdbbeffe82dbef183af9ffb404e89500d6bc0b6c3           8
3698  fdbfd653c850dfd6c4aea152f6500c52d5950e43d           9
3699  fdd14bdbd3f1aac3116c2c1169a86cbc61b1Ae9c1           0
3700  fdd658beec58b195a2c2edc5d92ddfca77285b559           9
3701  fde0cfe1e6656cd418c3f8a4e0ad47c4c17e25f34           1
3702  fdf35b62e7bd49fdf50bb2143bbaaf7121520ed65           8
3703  fe09e746b753fcee22e2c7d2be2883685761d4145          10
3704  fe255d4daf56461e5D7a5749a054eedc50b087509          10
3705  fe4e28d7b30d26a2fde0fc4c815e7eb0a52367fdd          10
3706  fe53725f47540d4f31d61bdd0a5aa5bb5cca28a50           8
3707  fe7887a3ff58c609fe633b6118e0c033a92ffd555           8
3708  feae6E291a49da14d05a6ffb9fbee2815d7c8153d           8
3709  fec6d2Eb64b083c0b8f4b4f0bd04671e78607be09           8
3710  fec8e57b083bd354dd6b02d129baae82f7591eae6           0
3711  fed5e724a012c38ec624fba4fd3650da59a5595c5           8
3712  feea3c1b728377170a6ad93689f0d9b2c07b0fe12          12
3713  ff15577c4d58442b758e85670263ada014fc91dd7           8
3714  ff272446010174efce45afa0e3ec77acba122b6df          10
3715  ff31496ea9aa36804397160dcc442890ec3a7df95          10
3716  ff393ca012e3bb6b89846114181ece70fde991c58           8
3717  ff3f10990b37a97c4990cfa36c14A6feb79c5ed32           8
3718  ff42c4ecb06a8838228a03cb930e87e3bdf5c08e2           8
3719  ff5abc2369a9Bea144b697fa2c147f37f8bbcc410          14
3720  ff7cc6e8d270894363730c008d579197bebcbd07d           8
3721  ff85866b215233b2fecdca2c2b8fda54ad24c86fd          12
3722  ff884224571e0476990574df5da76e0991db583af           8
3723  ffc47163a530c51ef2e6572d786aefbaed99890f2           8

[3724 rows x 2 columns]

In [248]:
print "Wrong classifications:", np.count_nonzero(y_valid-result), " of ", len(y_valid-result)

Wrong classifications: 66  of  617


In [172]:
train_centroids

array([[  7.81000000e+02,   1.18200000e+03,   7.50000000e+01, ...,
          1.04300000e+03,   1.37700000e+03,   5.30000000e+01],
       [  2.06230000e+04,   1.20470000e+04,   7.22660000e+04, ...,
          8.12000000e+03,   5.84200000e+03,   1.07500000e+03],
       [  9.90000000e+01,   2.34000000e+02,   1.30000000e+01, ...,
          5.00000000e+00,   2.70000000e+02,   4.00000000e+00],
       ..., 
       [  7.84000000e+02,   1.19500000e+03,   3.20000000e+01, ...,
          1.04900000e+03,   1.36500000e+03,   4.70000000e+01],
       [  8.49000000e+02,   3.35000000e+02,   7.80000000e+02, ...,
          1.44000000e+03,   1.59800000e+03,   1.02000000e+02],
       [  3.46000000e+02,   9.40000000e+01,   6.00000000e+00, ...,
          5.63000000e+02,   4.97000000e+02,   5.50000000e+01]], dtype=float32)